# Pinsage Pytorch 实现

In [13]:
import os

#全局配置
input_dir = "../../../data/ml-1m/"
output_dir = "../../../data/ml-1m/tmp/"
os.makedirs(output_dir, exist_ok=True)
os.getcwd()

# 模型运行参数
num_epochs = 3
num_workers = 2

hidden_dims = 64
batch_size = 32
num_random_walks=10
random_walk_length=2
random_walk_restart_prob=0.5
num_neighbors=3
num_layers=2
lr=3e-5
k=10
batches_per_epoch=100
device='cpu'

## 生成图样本文件
### Utils
##### 图处理工具

In [1]:
"""build.py Graph builder from pandas dataframes"""

from collections import namedtuple

import dgl
from pandas.api.types import is_categorical, is_categorical_dtype, is_numeric_dtype


def _series_to_tensor(serises):
    if is_categorical(series):
        return torch.LongTensor(series.cat.codes.values.astype("int64"))
    else:
        return torch.FloatTensor(series.values)


class PandasGraphBUilder(object):
    """Creates a heterogeneous graph from multiple pandas dataframes.
    Examples
    --------
    Let's say we have the following three pandas dataframes:
    User table ``users``:
    ===========  ===========  =======
    ``user_id``  ``country``  ``age``
    ===========  ===========  =======
    XYZZY        U.S.         25
    FOO          China        24
    BAR          China        23
    ===========  ===========  =======
    Game table ``games``:
    ===========  =========  ==============  ==================
    ``game_id``  ``title``  ``is_sandbox``  ``is_multiplayer``
    ===========  =========  ==============  ==================
    1            Minecraft  True            True
    2            Tetris 99  False           True
    ===========  =========  ==============  ==================
    Play relationship table ``plays``:
    ===========  ===========  =========
    ``user_id``  ``game_id``  ``hours``
    ===========  ===========  =========
    XYZZY        1            24
    FOO          1            20
    FOO          2            16
    BAR          2            28
    ===========  ===========  =========
    One could then create a bidirectional bipartite graph as follows:
    >>> builder = PandasGraphBuilder()
    >>> builder.add_entities(users, 'user_id', 'user')
    >>> builder.add_entities(games, 'game_id', 'game')
    >>> builder.add_binary_relations(plays, 'user_id', 'game_id', 'plays')
    >>> builder.add_binary_relations(plays, 'game_id', 'user_id', 'played-by')
    >>> g = builder.build()
    >>> g.num_nodes('user')
    3
    >>> g.num_edges('plays')
    4
    """

    def __init__(self):
        self.entity_tables = {}
        self.relation_tables = {}

        self.entity_pk_to_name = {}  # mapping from primary key name to entity name
        self.entity_pk = {}  # mapping from entity name to primary key
        self.entity_key_map = {}  # mapping from entity names to primary key values
        self.num_nodes_per_type = {}
        self.edges_per_relation = {}
        self.relation_src_key = {}
        self.relation_dst_key = {}
        self.relation_name_to_etype = {}

    def add_entities(self, entity_table, primary_key, name):
        entities = entity_table[primary_key].astype("category")
        if not (entities.value_counts() == 1).all():
            raise ValueError("Different entity with the same primary key detected.")
        # preserve the category order in the original entity table
        entities = entities.cat.reorder_categories(entity_table[primary_key].values)

        self.entity_pk_to_name[primary_key] = name
        self.entity_pk[name] = primary_key
        self.num_nodes_per_type[name] = entity_table.shape[0]
        self.entity_key_map[name] = entities
        self.entity_tables[name] = entity_table

    def add_binary_relations(self, relation_table, source_key, destination_key, name):
        src = relation_table[source_key].astype("category")
        src = src.cat.set_categories(
            self.entity_key_map[self.entity_pk_to_name[source_key]].cat.categories
        )
        dst = relation_table[destination_key].astype("category")
        dst = dst.cat.set_categories(
            self.entity_key_map[self.entity_pk_to_name[destination_key]].cat.categories
        )

        if src.isnull().any():
            raise ValueError(
                "Some source entities in relation %s do not exist in entity %s."
                % (name, source_key)
            )
        if dst.isnull().any():
            raise ValueError(
                "Some destination entities in relation %s do not exist in entity %s."
                % (name, destination_key)
            )

        srctype = self.entity_pk_to_name[source_key]
        dsttype = self.entity_pk_to_name[destination_key]
        etype = (srctype, name, dsttype)
        self.relation_name_to_etype[name] = etype
        self.edges_per_relation[etype] = (
            src.cat.codes.values.astype("int64"),
            dst.cat.codes.values.astype("int64"),
        )
        self.relation_tables[name] = relation_table
        self.relation_src_key[name] = source_key
        self.relation_dst_key[name] = destination_key

    def build(self):
        # Create heterograph
        graph = dgl.heterograph(self.edges_per_relation, self.num_nodes_per_type)
        return graph

In [13]:
import dask.dataframe as dd
import dgl
import numpy as np
import scipy.sparse as ssp
import torch
import tqdm


# This is the train-test split method most of the recommender system papers running on MovieLens
# takes.  It essentially follows the intuition of "training on the past and predict the future".
# One can also change the threshold to make validation and test set take larger proportions.
def train_test_split_by_time(df, timestamp, user):
    df["train_mask"] = np.ones((len(df),), dtype=np.bool)
    df["val_mask"] = np.zeros((len(df),), dtype=np.bool)
    df["test_mask"] = np.zeros((len(df),), dtype=np.bool)
    df = dd.from_pandas(df, npartitions=10)

    def train_test_split(df):
        df = df.sort_values([timestamp])
        if df.shape[0] > 1:
            df.iloc[-1, -3] = False
            df.iloc[-1, -1] = True
        if df.shape[0] > 2:
            df.iloc[-2, -3] = False
            df.iloc[-2, -2] = True
        return df
    print(type(df), df.shape,df.shape[0])
    df = (
        df.groupby(user, group_keys=False)
        .apply(train_test_split)
        .compute(scheduler="processes")
        .sort_index()
    )
    print(df[df[user] == df[user].unique()[0]].sort_values(timestamp))
    return (
        df["train_mask"].to_numpy().nonzero()[0],
        df["val_mask"].to_numpy().nonzero()[0],
        df["test_mask"].to_numpy().nonzero()[0],
    )


def build_train_graph(g, train_indices, utype, itype, etype, etype_rev):
    train_g = g.edge_subgraph(
        {etype: train_indices, etype_rev: train_indices}, relabel_nodes=False
    )

    # copy features
    for ntype in g.ntypes:
        for col, data in g.nodes[ntype].data.items():
            train_g.nodes[ntype].data[col] = data
    for etype in g.etypes:
        for col, data in g.edges[etype].data.items():
            train_g.edges[etype].data[col] = data[train_g.edges[etype].data[dgl.EID]]

    return train_g


def build_val_test_matrix(g, val_indices, test_indices, utype, itype, etype):
    n_users = g.num_nodes(utype)
    n_items = g.num_nodes(itype)
    val_src, val_dst = g.find_edges(val_indices, etype=etype)
    test_src, test_dst = g.find_edges(test_indices, etype=etype)
    val_src = val_src.numpy()
    val_dst = val_dst.numpy()
    test_src = test_src.numpy()
    test_dst = test_dst.numpy()
    val_matrix = ssp.coo_matrix(
        (np.ones_like(val_src), (val_src, val_dst)), (n_users, n_items)
    )
    test_matrix = ssp.coo_matrix(
        (np.ones_like(test_src), (test_src, test_dst)), (n_users, n_items)
    )

    return val_matrix, test_matrix


def linear_normalize(values):
    return (values - values.min(0, keepdims=True)) / (
        values.max(0, keepdims=True) - values.min(0, keepdims=True)
    )

### Prepare datasets（MovieLens 1M）
+ [dgl pinsage](https://github.com/dmlc/dgl/tree/master/examples/pytorch/pinsage)

MovieLens 1M数据集含有来自6000名用户对4000部电影的100万条评分数据。分为三个表：评分，用户信息，电影信息。这些数据都是dat文件格式。

In [3]:
import argparse
import os
import pickle
import re

import dgl
import numpy as np
import pandas as pd
import scipy.sparse as ssp
import torch
import torchtext

In [5]:
### Build heterogenerous graph
# Load data
users = []
with open(os.path.join(input_dir, "users.dat"), encoding="latin1") as f:
    for l in f:
        id_, gender, age, occupation, zip_ = l.strip().split("::")
        users.append(
            {
                "user_id": int(id_),
                "gender": gender,
                "age": age,
                "occupation": occupation,
                "zip": zip_,
            }
        )
users = pd.DataFrame(users).astype("category")

movies = []
with open(os.path.join(input_dir, "movies.dat"), encoding="latin1") as f:
    for l in f:
        id_, title, genres = l.strip().split("::")
        genres_set = set(genres.split("|"))

        # extract year
        assert re.match(r".*\([0-9]{4}\)$", title)
        year = title[-5:-1]
        title = title[:-6].strip()

        data = {"movie_id": int(id_), "title": title, "year": year}
        for g in genres_set:
            data[g] = True
        movies.append(data)
movies = pd.DataFrame(movies).astype({"year": "category"})

ratings = []
with open(os.path.join(input_dir, "ratings.dat"), encoding="latin1") as f:
    for l in f:
        user_id, movie_id, rating, timestamp = [int(_) for _ in l.split("::")]
        ratings.append(
            {
                "user_id": user_id,
                "movie_id": movie_id,
                "rating": rating,
                "timestamp": timestamp,
            }
        )
ratings = pd.DataFrame(ratings)

In [6]:
print("users:", users.shape)
print(users.head())
print("movies", movies.shape)
print(movies.head())
print("rating", ratings.shape)
print(ratings.head())

users: (6040, 5)
  user_id gender age occupation    zip
0       1      F   1         10  48067
1       2      M  56         16  70072
2       3      M  25         15  55117
3       4      M  45          7  02460
4       5      M  25         20  55455
movies (3883, 21)
   movie_id                        title  year Animation Comedy Children's  \
0         1                    Toy Story  1995      True   True       True   
1         2                      Jumanji  1995       NaN    NaN       True   
2         3             Grumpier Old Men  1995       NaN   True        NaN   
3         4            Waiting to Exhale  1995       NaN   True        NaN   
4         5  Father of the Bride Part II  1995       NaN   True        NaN   

  Adventure Fantasy Romance Drama  ... Crime Thriller Horror Sci-Fi  \
0       NaN     NaN     NaN   NaN  ...   NaN      NaN    NaN    NaN   
1      True    True     NaN   NaN  ...   NaN      NaN    NaN    NaN   
2       NaN     NaN    True   NaN  ...   NaN     

In [7]:
# Filter the users and items that never appear in the rating table.
distinct_users_in_rating = ratings["user_id"].unique()
distinct_movies_in_rating = ratings["movie_id"].unique()
users = users[users["user_id"].isin(distinct_users_in_rating)]
movies = movies[movies["movie_id"].isin(distinct_movies_in_rating)]

print("users:", users.shape)
print("movies", movies.shape)

users: (6040, 5)
movies (3706, 21)


In [8]:
# Group the movie features into genres (a vector), year (a category), title (a string)
genera_columns = movies.columns.drop(["movie_id", "title", "year"])
movies[genera_columns] = movies[genera_columns].fillna(False).astype("bool")
movies_categorical = movies.drop("title", axis=1)

print(movies_categorical.head())

   movie_id  year  Animation  Comedy  Children's  Adventure  Fantasy  Romance  \
0         1  1995       True    True        True      False    False    False   
1         2  1995      False   False        True       True     True    False   
2         3  1995      False    True       False      False    False     True   
3         4  1995      False    True       False      False    False    False   
4         5  1995      False    True       False      False    False    False   

   Drama  Action  Crime  Thriller  Horror  Sci-Fi  Documentary    War  \
0  False   False  False     False   False   False        False  False   
1  False   False  False     False   False   False        False  False   
2  False   False  False     False   False   False        False  False   
3   True   False  False     False   False   False        False  False   
4  False   False  False     False   False   False        False  False   

   Musical  Mystery  Film-Noir  Western  
0    False    False      False  

In [9]:
# Build graph
graph_builder = PandasGraphBUilder()
graph_builder.add_entities(users, "user_id", "user")
graph_builder.add_entities(movies_categorical, "movie_id", "movie")
graph_builder.add_binary_relations(ratings, "user_id", "movie_id", "watched")
graph_builder.add_binary_relations(ratings, "movie_id", "user_id", "watched-by")
g = graph_builder.build()

In [10]:
# Assign features.
# Note that variable-sized features such as texts or images are handled elsewhere.
g.nodes["user"].data["gender"] = torch.LongTensor(users["gender"].cat.codes.values)
g.nodes["user"].data["age"] = torch.LongTensor(users["age"].cat.codes.values)
g.nodes["user"].data["occupation"] = torch.LongTensor(
    users["occupation"].cat.codes.values
)
g.nodes["user"].data["zip"] = torch.LongTensor(users["zip"].cat.codes.values)

g.nodes["movie"].data["year"] = torch.LongTensor(movies["year"].cat.codes.values)
g.nodes["movie"].data["genre"] = torch.FloatTensor(movies[genera_columns].values)

g.edges["watched"].data["rating"] = torch.LongTensor(ratings["rating"].values)
g.edges["watched"].data["timestamp"] = torch.LongTensor(ratings["timestamp"].values)
g.edges["watched-by"].data["rating"] = torch.LongTensor(ratings["rating"].values)
g.edges["watched-by"].data["timestamp"] = torch.LongTensor(ratings["timestamp"].values)

/tmp/ipykernel_62828/887568951.py:3: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:178.)
  g.nodes["user"].data["gender"] = torch.LongTensor(users["gender"].cat.codes.values)


In [11]:
print((g.nodes["user"].data["gender"].shape))

torch.Size([6040])


In [15]:
# Train-validation-test split
# This is a little bit tricky as we want to select the last interaction for test, and the
# second-to-last interaction for validation.
train_indices, val_indices, test_indices = train_test_split_by_time(
    ratings, "timestamp", "user_id"
)
# Build the graph with training interactions only.
train_g = build_train_graph(g, train_indices, "user", "movie", "watched", "watched-by")
assert train_g.out_degrees(etype="watched").min() > 0

val_matrix, test_matrix = build_val_test_matrix(
    g, val_indices, test_indices, "user", "movie", "watched"
)
## Build title set
movie_textual_dataset = {"title": movies["title"].values}
# The model should build their own vocabulary and process the texts.  Here is one example
# of using torchtext to pad and numericalize a batch of strings.
#     field = torchtext.data.Field(include_lengths=True, lower=True, batch_first=True)
#     examples = [torchtext.data.Example.fromlist([t], [('title', title_field)]) for t in texts]
#     titleset = torchtext.data.Dataset(examples, [('title', title_field)])
#     field.build_vocab(titleset.title, vectors='fasttext.simple.300d')
#     token_ids, lengths = field.process([examples[0].title, examples[1].title])
## Dump the graph and the datasets
dgl.save_graphs(os.path.join(output_dir, "train_g.bin"), train_g)
dataset = {
    "val-matrix": val_matrix,
    "test-matrix": test_matrix,
    "item-texts": movie_textual_dataset,
    "item-images": None,
    "user-type": "user",
    "item-type": "movie",
    "user-to-item-type": "watched",
    "item-to-user-type": "watched-by",
    "timestamp-edge-column": "timestamp",
}
with open(os.path.join(output_dir, "data.pkl"), "wb") as f:
    pickle.dump(dataset, f)

<class 'dask.dataframe.core.DataFrame'> (Delayed('int-c845e192-8e59-4b84-8f62-db71a84079b1'), 7) Delayed('int-32a66189-d86e-468c-a19e-e64c74b4f8c1')


/tmp/ipykernel_62828/680607132.py:13: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df["train_mask"] = np.ones((len(df),), dtype=np.bool)
/tmp/ipykernel_62828/680607132.py:14: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df["val_mask"] = np.zeros((len(df),), dtype=np.bool)
/tmp/ipykernel_62828/680607132.py:15: DeprecationWarning: `np.bool` is a de

    user_id  movie_id  rating  timestamp  train_mask  val_mask  test_mask
31        1      3186       4  978300019        True     False      False
27        1      1721       4  978300055        True     False      False
37        1      1022       5  978300055        True     False      False
22        1      1270       5  978300055        True     False      False
24        1      2340       3  978300103        True     False      False
36        1      1836       5  978300172        True     False      False
3         1      3408       4  978300275        True     False      False
47        1      1207       4  978300719        True     False      False
7         1      2804       5  978300719        True     False      False
21        1       720       3  978300760        True     False      False
0         1      1193       5  978300760        True     False      False
44        1       260       4  978300760        True     False      False
9         1       919       4  9783013

## 准备模型训练

In [2]:
import pickle
import argparse
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchtext
import dgl
import os
import tqdm
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [3]:

data_info_path = os.path.join(output_dir, 'data.pkl')
with open(data_info_path, 'rb') as f:
    dataset = pickle.load(f)
train_g_path = os.path.join(output_dir, 'train_g.bin')
g_list, _ = dgl.load_graphs(train_g_path)
dataset['train-graph'] = g_list[0]


In [4]:
print(type(dataset), dataset.keys())
print(type(g_list), dataset['train-graph'] )

<class 'dict'> dict_keys(['val-matrix', 'test-matrix', 'item-texts', 'item-images', 'user-type', 'item-type', 'user-to-item-type', 'item-to-user-type', 'timestamp-edge-column', 'train-graph'])
<class 'list'> Graph(num_nodes={'movie': 3706, 'user': 6040},
      num_edges={('movie', 'watched-by', 'user'): 988129, ('user', 'watched', 'movie'): 988129},
      metagraph=[('movie', 'user', 'watched-by'), ('user', 'movie', 'watched')])


### Sampler

In [16]:
import numpy as np
import dgl
import torch
from torch.utils.data import IterableDataset, DataLoader
from torchtext.data.functional import numericalize_tokens_from_iterator

def padding(array, yy, val):
    """
    :param array: torch tensor array
    :param yy: desired width
    :param val: padded value
    :return: padded array
    """
    w = array.shape[0]
    b = 0
    bb = yy - b - w

    return torch.nn.functional.pad(array, pad=(b, bb), mode='constant', value=val)

def compact_and_copy(frontier, seeds):
    block = dgl.to_block(frontier, seeds)
    for col, data in frontier.edata.items():
        if col == dgl.EID:
            continue
        block.edata[col] = data[block.edata[dgl.EID]]
    return block

class ItemToItemBatchSampler(IterableDataset):
    def __init__(self, g, user_type, item_type, batch_size):
        self.g = g
        self.user_type = user_type
        self.item_type = item_type
        self.user_to_item_etype = list(g.metagraph()[user_type][item_type])[0]
        self.item_to_user_etype = list(g.metagraph()[item_type][user_type])[0]
        self.batch_size = batch_size

    def __iter__(self):
        while True:
            heads = torch.randint(0, self.g.num_nodes(self.item_type), (self.batch_size,))
            tails = dgl.sampling.random_walk(
                self.g,
                heads,
                metapath=[self.item_to_user_etype, self.user_to_item_etype])[0][:, 2]
            neg_tails = torch.randint(0, self.g.num_nodes(self.item_type), (self.batch_size,))

            mask = (tails != -1)
            yield heads[mask], tails[mask], neg_tails[mask]

class NeighborSampler(object):
    def __init__(self, g, user_type, item_type, random_walk_length, random_walk_restart_prob,
                 num_random_walks, num_neighbors, num_layers):
        self.g = g
        self.user_type = user_type
        self.item_type = item_type
        self.user_to_item_etype = list(g.metagraph()[user_type][item_type])[0]
        self.item_to_user_etype = list(g.metagraph()[item_type][user_type])[0]
        self.samplers = [
            dgl.sampling.PinSAGESampler(g, item_type, user_type, random_walk_length,
                random_walk_restart_prob, num_random_walks, num_neighbors)
            for _ in range(num_layers)]

    def sample_blocks(self, seeds, heads=None, tails=None, neg_tails=None):
        blocks = []
        for sampler in self.samplers:
            frontier = sampler(seeds)
            if heads is not None:
                eids = frontier.edge_ids(torch.cat([heads, heads]), torch.cat([tails, neg_tails]), return_uv=True)[2]
                if len(eids) > 0:
                    old_frontier = frontier
                    frontier = dgl.remove_edges(old_frontier, eids)
                    #print(old_frontier)
                    #print(frontier)
                    #print(frontier.edata['weights'])
                    #frontier.edata['weights'] = old_frontier.edata['weights'][frontier.edata[dgl.EID]]
            block = compact_and_copy(frontier, seeds)
            seeds = block.srcdata[dgl.NID]
            blocks.insert(0, block)
        return blocks

    def sample_from_item_pairs(self, heads, tails, neg_tails):
        # Create a graph with positive connections only and another graph with negative
        # connections only.
        pos_graph = dgl.graph(
            (heads, tails),
            num_nodes=self.g.num_nodes(self.item_type))
        neg_graph = dgl.graph(
            (heads, neg_tails),
            num_nodes=self.g.num_nodes(self.item_type))
        pos_graph, neg_graph = dgl.compact_graphs([pos_graph, neg_graph])
        seeds = pos_graph.ndata[dgl.NID]

        blocks = self.sample_blocks(seeds, heads, tails, neg_tails)
        return pos_graph, neg_graph, blocks

def assign_simple_node_features(ndata, g, ntype, assign_id=False):
    """
    Copies data to the given block from the corresponding nodes in the original graph.
    """
    for col in g.nodes[ntype].data.keys():
        if not assign_id and col == dgl.NID:
            continue
        induced_nodes = ndata[dgl.NID]
        ndata[col] = g.nodes[ntype].data[col][induced_nodes]

def assign_textual_node_features(ndata, textset, ntype):
    """
    Assigns numericalized tokens from a torchtext dataset to given block.
    The numericalized tokens would be stored in the block as node features
    with the same name as ``field_name``.
    The length would be stored as another node feature with name
    ``field_name + '__len'``.
    block : DGLHeteroGraph
        First element of the compacted blocks, with "dgl.NID" as the
        corresponding node ID in the original graph, hence the index to the
        text dataset.
        The numericalized tokens (and lengths if available) would be stored
        onto the blocks as new node features.
    textset : torchtext.data.Dataset
        A torchtext dataset whose number of examples is the same as that
        of nodes in the original graph.
    """
    node_ids = ndata[dgl.NID].numpy()

    for field_name, field in textset.items():
        textlist, vocab, pad_var, batch_first = field
        
        examples = [textlist[i] for i in node_ids]
        ids_iter = numericalize_tokens_from_iterator(vocab, examples)
        
        maxsize = max([len(textlist[i]) for i in node_ids])
        ids = next(ids_iter)
        x = torch.asarray([num for num in ids])
        lengths = torch.tensor([len(x)])
        tokens = padding(x, maxsize, pad_var)

        for ids in ids_iter:
            x = torch.asarray([num for num in ids])
            l = torch.tensor([len(x)])
            y = padding(x, maxsize, pad_var)
            tokens = torch.vstack((tokens,y))
            lengths = torch.cat((lengths, l))
       
        if not batch_first:
            tokens = tokens.t()

        ndata[field_name] = tokens
        ndata[field_name + '__len'] = lengths

def assign_features_to_blocks(blocks, g, textset, ntype):
    # For the first block (which is closest to the input), copy the features from
    # the original graph as well as the texts.
    assign_simple_node_features(blocks[0].srcdata, g, ntype)
    assign_textual_node_features(blocks[0].srcdata, textset, ntype)
    assign_simple_node_features(blocks[-1].dstdata, g, ntype)
    assign_textual_node_features(blocks[-1].dstdata, textset, ntype)

class PinSAGECollator(object):
    def __init__(self, sampler, g, ntype, textset):
        self.sampler = sampler
        self.ntype = ntype
        self.g = g
        self.textset = textset

    def collate_train(self, batches):
        heads, tails, neg_tails = batches[0]
        # Construct multilayer neighborhood via PinSAGE...
        pos_graph, neg_graph, blocks = self.sampler.sample_from_item_pairs(heads, tails, neg_tails)
        assign_features_to_blocks(blocks, self.g, self.textset, self.ntype)

        return pos_graph, neg_graph, blocks

    def collate_test(self, samples):
        batch = torch.LongTensor(samples)
        blocks = self.sampler.sample_blocks(batch)
        assign_features_to_blocks(blocks, self.g, self.textset, self.ntype)
        return blocks

### 定义模型结构

##### Layer定义

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
import dgl.nn.pytorch as dglnn
import dgl.function as fn

def disable_grad(module):
    for param in module.parameters():
        param.requires_grad = False

def _init_input_modules(g, ntype, textset, hidden_dims):
    # We initialize the linear projections of each input feature ``x`` as
    # follows:
    # * If ``x`` is a scalar integral feature, we assume that ``x`` is a categorical
    #   feature, and assume the range of ``x`` is 0..max(x).
    # * If ``x`` is a float one-dimensional feature, we assume that ``x`` is a
    #   numeric vector.
    # * If ``x`` is a field of a textset, we process it as bag of words.
    module_dict = nn.ModuleDict()

    for column, data in g.nodes[ntype].data.items():
        if column == dgl.NID:
            continue
        if data.dtype == torch.float32:
            assert data.ndim == 2
            m = nn.Linear(data.shape[1], hidden_dims)
            nn.init.xavier_uniform_(m.weight)
            nn.init.constant_(m.bias, 0)
            module_dict[column] = m
        elif data.dtype == torch.int64:
            assert data.ndim == 1
            m = nn.Embedding(
                data.max() + 2, hidden_dims, padding_idx=-1)
            nn.init.xavier_uniform_(m.weight)
            module_dict[column] = m

    if textset is not None:
        for column, field in textset.items():
            textlist, vocab, pad_var, batch_first = field            
            module_dict[column] = BagOfWords(vocab, hidden_dims)

    return module_dict

class BagOfWords(nn.Module):
    def __init__(self, vocab, hidden_dims):
        super().__init__()

        self.emb = nn.Embedding(
            len(vocab.get_itos()), hidden_dims,
            padding_idx=vocab.get_stoi()['<pad>'])
        nn.init.xavier_uniform_(self.emb.weight)

    def forward(self, x, length):
        return self.emb(x).sum(1) / length.unsqueeze(1).float()

class LinearProjector(nn.Module):
    """
    Projects each input feature of the graph linearly and sums them up
    """
    def __init__(self, full_graph, ntype, textset, hidden_dims):
        super().__init__()

        self.ntype = ntype
        self.inputs = _init_input_modules(full_graph, ntype, textset, hidden_dims)

    def forward(self, ndata):
        projections = []
        for feature, data in ndata.items():
            if feature == dgl.NID or feature.endswith('__len'):
                # This is an additional feature indicating the length of the ``feature``
                # column; we shouldn't process this.
                continue

            module = self.inputs[feature]
            if isinstance(module, BagOfWords):
                # Textual feature; find the length and pass it to the textual module.
                length = ndata[feature + '__len']
                result = module(data, length)
            else:
                result = module(data)
            projections.append(result)

        return torch.stack(projections, 1).sum(1)

class WeightedSAGEConv(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims, act=F.relu):
        super().__init__()

        self.act = act
        self.Q = nn.Linear(input_dims, hidden_dims)
        self.W = nn.Linear(input_dims + hidden_dims, output_dims)
        self.reset_parameters()
        self.dropout = nn.Dropout(0.5)

    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_uniform_(self.Q.weight, gain=gain)
        nn.init.xavier_uniform_(self.W.weight, gain=gain)
        nn.init.constant_(self.Q.bias, 0)
        nn.init.constant_(self.W.bias, 0)

    def forward(self, g, h, weights):
        """
        g : graph
        h : node features
        weights : scalar edge weights
        """
        h_src, h_dst = h
        with g.local_scope():
            g.srcdata['n'] = self.act(self.Q(self.dropout(h_src)))
            g.edata['w'] = weights.float()
            g.update_all(fn.u_mul_e('n', 'w', 'm'), fn.sum('m', 'n'))
            g.update_all(fn.copy_e('w', 'm'), fn.sum('m', 'ws'))
            n = g.dstdata['n']
            ws = g.dstdata['ws'].unsqueeze(1).clamp(min=1)
            z = self.act(self.W(self.dropout(torch.cat([n / ws, h_dst], 1))))
            z_norm = z.norm(2, 1, keepdim=True)
            z_norm = torch.where(z_norm == 0, torch.tensor(1.).to(z_norm), z_norm)
            z = z / z_norm
            return z

class SAGENet(nn.Module):
    def __init__(self, hidden_dims, n_layers):
        """
        g : DGLHeteroGraph
            The user-item interaction graph.
            This is only for finding the range of categorical variables.
        item_textsets : torchtext.data.Dataset
            The textual features of each item node.
        """
        super().__init__()

        self.convs = nn.ModuleList()
        for _ in range(n_layers):
            self.convs.append(WeightedSAGEConv(hidden_dims, hidden_dims, hidden_dims))

    def forward(self, blocks, h):
        for layer, block in zip(self.convs, blocks):
            h_dst = h[:block.num_nodes('DST/' + block.ntypes[0])]
            h = layer(block, (h, h_dst), block.edata['weights'])
        return h

class ItemToItemScorer(nn.Module):
    def __init__(self, full_graph, ntype):
        super().__init__()

        n_nodes = full_graph.num_nodes(ntype)
        self.bias = nn.Parameter(torch.zeros(n_nodes, 1))

    def _add_bias(self, edges):
        bias_src = self.bias[edges.src[dgl.NID]]
        bias_dst = self.bias[edges.dst[dgl.NID]]
        return {'s': edges.data['s'] + bias_src + bias_dst}

    def forward(self, item_item_graph, h):
        """
        item_item_graph : graph consists of edges connecting the pairs
        h : hidden state of every node
        """
        with item_item_graph.local_scope():
            item_item_graph.ndata['h'] = h
            item_item_graph.apply_edges(fn.u_dot_v('h', 'h', 's'))
            item_item_graph.apply_edges(self._add_bias)
            pair_score = item_item_graph.edata['s']
        return pair_score

##### 模型定义

In [9]:
class PinSAGEModel(nn.Module):
    def __init__(self, full_graph, ntype, textsets, hidden_dims, n_layers):
        super().__init__()

        self.proj = LinearProjector(full_graph, ntype, textsets, hidden_dims)
        self.sage = SAGENet(hidden_dims, n_layers)
        self.scorer = ItemToItemScorer(full_graph, ntype)

    def forward(self, pos_graph, neg_graph, blocks):
        h_item = self.get_repr(blocks)
        pos_score = self.scorer(pos_graph, h_item)
        neg_score = self.scorer(neg_graph, h_item)
        return (neg_score - pos_score + 1).clamp(min=0)

    def get_repr(self, blocks):
        h_item = self.proj(blocks[0].srcdata)
        h_item_dst = self.proj(blocks[-1].dstdata)
        return h_item_dst + self.sage(blocks, h_item)

##### Evaluation定义

In [11]:
import numpy as np
import torch
import pickle
import dgl
import argparse

def prec(recommendations, ground_truth):
    n_users, n_items = ground_truth.shape
    K = recommendations.shape[1]
    user_idx = np.repeat(np.arange(n_users), K)
    item_idx = recommendations.flatten()
    relevance = ground_truth[user_idx, item_idx].reshape((n_users, K))
    hit = relevance.any(axis=1).mean()
    return hit

class LatestNNRecommender(object):
    def __init__(self, user_ntype, item_ntype, user_to_item_etype, timestamp, batch_size):
        self.user_ntype = user_ntype
        self.item_ntype = item_ntype
        self.user_to_item_etype = user_to_item_etype
        self.batch_size = batch_size
        self.timestamp = timestamp

    def recommend(self, full_graph, K, h_user, h_item):
        """
        Return a (n_user, K) matrix of recommended items for each user
        """
        graph_slice = full_graph.edge_type_subgraph([self.user_to_item_etype])
        n_users = full_graph.num_nodes(self.user_ntype)
        latest_interactions = dgl.sampling.select_topk(graph_slice, 1, self.timestamp, edge_dir='out')
        user, latest_items = latest_interactions.all_edges(form='uv', order='srcdst')
        # each user should have at least one "latest" interaction
        assert torch.equal(user, torch.arange(n_users))

        recommended_batches = []
        user_batches = torch.arange(n_users).split(self.batch_size)
        for user_batch in user_batches:
            latest_item_batch = latest_items[user_batch].to(device=h_item.device)
            dist = h_item[latest_item_batch] @ h_item.t()
            # exclude items that are already interacted
            for i, u in enumerate(user_batch.tolist()):
                interacted_items = full_graph.successors(u, etype=self.user_to_item_etype)
                dist[i, interacted_items] = -np.inf
            recommended_batches.append(dist.topk(K, 1)[1])

        recommendations = torch.cat(recommended_batches, 0)
        return recommendations


def evaluate_nn(dataset, h_item, k, batch_size):
    g = dataset['train-graph']
    val_matrix = dataset['val-matrix'].tocsr()
    test_matrix = dataset['test-matrix'].tocsr()
    item_texts = dataset['item-texts']
    user_ntype = dataset['user-type']
    item_ntype = dataset['item-type']
    user_to_item_etype = dataset['user-to-item-type']
    timestamp = dataset['timestamp-edge-column']

    rec_engine = LatestNNRecommender(
        user_ntype, item_ntype, user_to_item_etype, timestamp, batch_size)

    recommendations = rec_engine.recommend(g, k, None, h_item).cpu().numpy()
    return prec(recommendations, val_matrix)

In [17]:
g= dataset['train-graph']
val_matrix = dataset['val-matrix'].tocsr()
test_matrix = dataset['test-matrix'].tocsr()
item_texts = dataset['item-texts']
user_ntype = dataset['user-type']
item_ntype = dataset['item-type']
user_to_item_etype = dataset['user-to-item-type']
timestamp = dataset['timestamp-edge-column']
device = torch.device(device)
# Assign user and movie IDs and use them as features (to learn an individual trainable
# embedding for each entity)
g.nodes[user_ntype].data['id'] = torch.arange(g.num_nodes(user_ntype))
g.nodes[item_ntype].data['id'] = torch.arange(g.num_nodes(item_ntype))
# Prepare torchtext dataset and Vocabulary
textset = {}
tokenizer = get_tokenizer(None)
textlist = []
batch_first = True
for i in range(g.num_nodes(item_ntype)):
    for key in item_texts.keys():
        l = tokenizer(item_texts[key][i].lower())
        textlist.append(l)
for key, field in item_texts.items():
    vocab2 = build_vocab_from_iterator(textlist, specials=["<unk>","<pad>"])
    textset[key] = (textlist, vocab2, vocab2.get_stoi()['<pad>'], batch_first)
# Sampler
batch_sampler = ItemToItemBatchSampler(
    g, user_ntype, item_ntype, batch_size)
neighbor_sampler = NeighborSampler(
    g, user_ntype, item_ntype, random_walk_length,
    random_walk_restart_prob, num_random_walks, num_neighbors,
    num_layers)
collator = PinSAGECollator(neighbor_sampler, g, item_ntype, textset)
dataloader = DataLoader(
    batch_sampler,
    collate_fn=collator.collate_train,
    num_workers=num_workers)
dataloader_test = DataLoader(
    torch.arange(g.num_nodes(item_ntype)),
    batch_size=batch_size,
    collate_fn=collator.collate_test,
    num_workers=num_workers)
dataloader_it = iter(dataloader)
# Model
model = PinSAGEModel(g, item_ntype, textset,hidden_dims, num_layers).to(device)
# Optimizer
opt = torch.optim.Adam(model.parameters(), lr=lr)
# For each batch of head-tail-negative triplets...
for epoch_id in range(num_epochs):
    model.train()
    for batch_id in tqdm.trange(batches_per_epoch):
        pos_graph, neg_graph, blocks = next(dataloader_it)
        # Copy to GPU
        for i in range(len(blocks)):
            blocks[i] = blocks[i].to(device)
        pos_graph = pos_graph.to(device)
        neg_graph = neg_graph.to(device)
        loss = model(pos_graph, neg_graph, blocks).mean()
        opt.zero_grad()
        loss.backward()
        opt.step()
    # Evaluate
    model.eval()
    with torch.no_grad():
        item_batches = torch.arange(g.num_nodes(item_ntype)).split(batch_size)
        h_item_batches = []
        for blocks in dataloader_test:
            for i in range(len(blocks)):
                blocks[i] = blocks[i].to(device)
            h_item_batches.append(model.get_repr(blocks))
        h_item = torch.cat(h_item_batches, 0)
        print(evaluate_nn(dataset, h_item, k, batch_size))


100%|██████████| 100/100 [00:17<00:00,  5.88it/s]


0.03642384105960265


100%|██████████| 100/100 [00:15<00:00,  6.32it/s]


0.03658940397350993


100%|██████████| 100/100 [00:15<00:00,  6.45it/s]


0.0390728476821192
